# Lotka-Volterra simulator: joint ratio variance vs theta distance

In [1]:
import numpy as np
import corner
from matplotlib import pyplot as plt
%matplotlib inline
import logging
import matplotlib

from goldmine.simulators.lotka_volterra import LotkaVolterra

logging.basicConfig(format='%(asctime)s  %(message)s', datefmt='%H:%M')
logging.getLogger().setLevel(logging.DEBUG)

## Run simulator and collect log r, t

In [2]:
simulator = LotkaVolterra()
theta0 = simulator.theta_defaults(single_theta=True)[0][0]
print(theta0)

[-4.60517019 -0.69314718  0.         -4.60517019]


In [ ]:
n_thetas = 10
n_samples = 10

In [ ]:
dthetas = []
log_r_xz = []
t_xz = []

distances = np.exp(np.linspace(np.log(0.01), np.log(1.), n_thetas))
r_variances = []

for i_theta, distance in enumerate(distances):
    dtheta = np.random.rand(4)
    dtheta *= distance / np.linalg.norm(dtheta)
    theta1 = theta0 + dtheta
    
    logging.info('\n\n--------------------------------------------------------------------------------\n\n distance = %s: delta theta = %s\n\n--------------------------------------------------------------------------------\n', distance, dtheta)
    
    x, this_r_xz, this_t_xz = simulator.rvs_ratio_score(theta0, theta1, theta0, theta0, n_samples)
    
    logging.info('\n\nResults: log r = %s, t = %s\n\n', np.log(this_r_xz), this_t_xz)
    
    log_r_xz += list(np.log(this_r_xz))
    t_xz += list(this_t_xz)
    dthetas += [dtheta] * len(this_r_xz)
    
    r_variances.append(np.var(this_r_xz))
    
log_r_xz = np.array(log_r_xz)
t_xz = np.array(t_xz)
dthetas = np.array(dthetas)

r_variances = np.array(r_variances)


10:51  

--------------------------------------------------------------------------------

 distance = 0.010000000000000004: delta theta = [0.00914648 0.00239007 0.00236673 0.00224232]

--------------------------------------------------------------------------------

10:51  Simulating 10 evolutions for theta = [-4.60517019 -0.69314718  0.         -4.60517019], augmenting with joint ratio between [-4.59602370e+00 -6.90757108e-01  2.36672983e-03 -4.60292787e+00] and [-4.60517019 -0.69314718  0.         -4.60517019] and joint score at  [-4.60517019 -0.69314718  0.         -4.60517019]
10:51    Starting sample 1 of 10
10:51    Starting sample 2 of 10
10:51    Starting sample 3 of 10
10:51    Starting sample 4 of 10
10:51    Starting sample 5 of 10


## Plot theta distance vs r variance

In [ ]:
fig = plt.figure(figsize=(4,4))
ax = plt.gca()

sc = plt.scatter(distances, r_variances, s=10.)

plt.xlabel(r'$\log r$')
plt.ylabel(r'$t \cdot \Delta \theta$')
#plt.xlim(0.008,1.2)
#plt.ylim(0.,1000.)
ax.set_xscale("log", nonposx='clip')
#ax.set_yscale("log", nonposy='clip')

plt.tight_layout()
#plt.savefig('lv_local_model_zoom.pdf')
plt.show()